In [1]:
from collections import defaultdict
import numpy as np
import random
import itertools
import re
import math

trainingSetTextFile = "dataset/training_set_clean_only_text.txt"
trainingSetTagsFile = "dataset/training_set_clean_only_tags.txt"

In [2]:
testData = []
tagsData = []
data = []

with open(trainingSetTextFile, mode='r', encoding='utf8') as f:
    # here we load learning data
    n = 1000000
    while n > 0:
        c = f.readline()
        if not c:
            print("End of file")
            break

        #c = c.lower()
        #c = re.split(SPLIT_STRING, c)
        #c = c.split()   
        #c = splitWords(c)
        
        testData.append(c)
        
        n -= 1
        
with open(trainingSetTagsFile, mode='r', encoding='utf8') as f:
    # here we load learning data
    n = 1000000
    while n > 0:
        c = f.readline()
        if not c:
            print("End of file")
            break

        #c = c.lower()
        #c = re.split(SPLIT_STRING, c)
        #c = c.split()   
        #c = splitWords(c)
        
        tagsData.append(c)
        
        n -= 1
        
for i in range(0, len(testData)):
    data.append((testData[i], int(tagsData[i])))

End of file
End of file


In [3]:
print(data)

[('Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.\n', 0), ('@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe\n', 0), ('@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.\n', 0), ('@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.\n', 0), ('Odrzut natychmiastowy, kwaśna mina, mam problem\n', 0), ('Jaki on był fajny xdd pamiętam, że spóźniłam się na jego pierwsze zajęcia i to sporo i za karę kazał mi usiąść w pierwszej ławce XD\n', 0), ('@anonymized_account No nie ma u nas szczęścia 😉\n', 0), ('@anonymized_account Dawno kogoś tak wrednego nie widziałam xd\n', 0), ('@anonymized_account @anonymized_account Zaległości były, ale ważne czy były wezwania do zapłaty z których się klub nie wywiązał.\n', 0), ('@anonymized_account @anonymized_account @anonymized_account Gdzie jest @anonymized_account . Brudziński jesteś kłamcą i marnym kutas

In [4]:
def split(sentence):
    words = sentence.split()
    return words

### Naive Bayes Classifier

In [6]:
wordClassifier = defaultdict(dict)

# preprocessing for each word
for i in range(0, len(data)):
    (sentence, res) = data[i]
    res = int(res)
    for word in split(sentence):
        if res == 0:
            if word in wordClassifier:
                wordClassifier[word][0] += 1
            else:
                wordClassifier[word] = [1, 0]
        else:
            if word in wordClassifier:
                wordClassifier[word][1] += 1
            else:
                wordClassifier[word] = [0, 1]
                
print(wordClassifier)
#print(wordClassifier["kutasem"])

defaultdict(<class 'dict'>, {'Dla': [25, 1], 'mnie': [231, 7], 'faworytem': [3, 0], 'do': [841, 67], 'tytułu': [2, 0], 'będzie': [267, 16], 'Cracovia.': [3, 0], 'Zobaczymy,': [2, 0], 'czy': [335, 22], 'typ': [1, 0], 'się': [1573, 126], 'sprawdzi.': [1, 0], '@anonymized_account': [12319, 1413], 'Brawo': [16, 0], 'ty': [79, 54], 'Daria': [4, 0], 'kibic': [4, 0], 'ma': [460, 40], 'być': [176, 17], 'na': [1774, 139], 'dobre': [23, 5], 'i': [1639, 176], 'złe': [6, 0], 'Super,': [2, 0], 'polski': [7, 1], 'premier': [3, 0], 'składa': [1, 0], 'kwiaty': [5, 0], 'grobach': [1, 0], 'kolaborantów.': [1, 0], 'Ale': [217, 14], 'doczekaliśmy': [1, 0], 'czasów.': [1, 0], 'Musi.': [1, 0], 'Innej': [2, 0], 'drogi': [3, 0], 'nie': [2354, 224], 'mamy.': [2, 0], 'Odrzut': [1, 0], 'natychmiastowy,': [1, 0], 'kwaśna': [1, 0], 'mina,': [1, 0], 'mam': [204, 1], 'problem': [25, 1], 'Jaki': [22, 4], 'on': [63, 7], 'był': [155, 14], 'fajny': [12, 0], 'xdd': [5, 0], 'pamiętam,': [3, 0], 'że': [1193, 89], 'spóźniła

In [7]:
countNeutral = 0
countHate = 0
pC = 0

for i in range(0, len(data)):
    (fst, res) = data[i]
    res = int(res)
    if res == 0:
        countNeutral += 1
    else:
        countHate += 1
        
pCNeutral = countNeutral / (countNeutral + countHate)
pCHate = countHate / (countNeutral + countHate)
print(countNeutral)
print(countHate)
print(pCNeutral)
print(pCHate)

# we should now count all p(W1|C)p(W2|C)...p(Wn|C)

9190
851
0.915247485310228
0.08475251468977194


In [24]:
def naivebayes(sentence):
    words = split(sentence)
    
    # we count log (P(C)*P(C|W1)*P(C|W2)*...*P(C|Wn)) = log (P(C)) + log P(C|W1) + log P(C|W2) + ... + log P(C|Wn)
    logSumNeutral = math.log(pCNeutral)
    logSumHate = math.log(pCHate)
    for word in words:
        (neutral, hate) = wordClassifier[word]
        if (neutral + hate != 0): # means we do have that in dictionary
            neutralProbability = (neutral) / (neutral + hate)
            hateProbability = (hate) / (neutral + hate)
            if (neutralProbability == 0):
                neutralProbability = 0.000000001
            
            if (hateProbability == 0):
                hateProbability = 0.000000001
            
            logSumNeutral += math.log(neutralProbability)
            logSumHate += math.log(hateProbability)
            #print(neutral)
            #print(hate)
            #print(neutralProbability)
            # find p(W1|C)
        else:
            print("The word has not been found in the dictionary!")
    
    probNeutral = math.exp(logSumNeutral)
    probHate = math.exp(logSumHate)
    # we want to rescale probability
    # we look for p1/p2 = x/y such that x + y = 1 
    y = probHate / (probNeutral + probHate)
    probNeutral = 1 - y
    probHate = y
    
    return (probNeutral, probHate)

In [25]:
test = "To jest test"
res = naivebayes(test)
print("Probability that: '" + test + "' is neutral: " + str(res))

Probability that: 'To jest test' is neutral: (0.9999999999991461, 8.538539229107636e-13)


In [26]:
result = []
for i in range(0, len(data)):
    (sentence, res) = data[i]
    prob = naivebayes(sentence)
    result.append(prob)
    
# we will shrink the result into 0, 1 classifier
#classifier = []
#for i in range(0, len(result)):
#    if (result[i] >= 0.5):
#        classifier.append(1)
#    else:
#        classifier.append(0)
        
classifier = result
        
for i in range(0, len(data)):
    (sent, tag) = data[i]
    print(sent + "|" + str(tag) + "|(" + str(classifier[i][0]) + ", " + str(classifier[i][1]) + ")")

Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.
|0|(1.0, 2.8188198727472167e-63)
@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe
|0|(1.0, 1.279350699774678e-44)
@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.
|0|(1.0, 8.797897714881138e-79)
@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.
|0|(1.0, 1.1592818107089392e-40)
Odrzut natychmiastowy, kwaśna mina, mam problem
|0|(1.0, 1.8156990761484373e-41)
Jaki on był fajny xdd pamiętam, że spóźniłam się na jego pierwsze zajęcia i to sporo i za karę kazał mi usiąść w pierwszej ławce XD
|0|(1.0, 7.870191667385817e-124)
@anonymized_account No nie ma u nas szczęścia 😉
|0|(1.0, 2.1040416296358307e-26)
@anonymized_account Dawno kogoś tak wrednego nie widziałam xd
|0|(1.0, 2.0534859481052217e-34)
@anonymized_account @anonymized_account Zaległości były, ale ważne czy były wezwania do zapł

|0|(1.0, 5.822571861323484e-41)
@anonymized_account No przecież, a osoby lgbt w ogóle nie korzystają z komunikacji miejskiej no nie, wiec oni nie są już pasażerami :) parodia
|0|(1.0, 2.8282332322764874e-107)
\"Polska to kraj z dykty i paździerza z postsowieckim społeczeństwem mającym demokrację za fanaberie zachodnich elit\" made by @anonymized_account
|0|(1.0, 1.1787436220405813e-108)
RT @anonymized_account \"Polska to kraj z dykty i paździerza z postsowieckim społeczeństwem mającym demokrację za fanaberie zachodnich elit\" made…
|0|(1.0, 3.0004383106487628e-108)
@anonymized_account @anonymized_account Co nie oznacza, że Morawiecki nie kłamał. Tradycyjnie łże na każdym kroku.
|0|(1.0, 1.5651072863562744e-45)
@anonymized_account będzie trenerem reprezentacji :) żona mu załatwi :)
|0|(1.0, 6.125603722224868e-45)
o chuj paweł w przyjaciółkach nie żyje?????
|0|(1.0, 2.3824679473303506e-32)
@anonymized_account Zegarmistrz precyzyjny polskiej polityki Grzegorz Schetyna pojechał do centrali

RT @anonymized_account @anonymized_account Zamknij niemiecki ryj,siedź w tej Brukseli i nie wracaj złodzieju!😬😬😬
|1|(0.0, 1.0)
@anonymized_account @anonymized_account należe do tej grupy osób :)
|0|(1.0, 7.801998967448679e-35)
@anonymized_account @anonymized_account Brawo @anonymized_account !!!! Kolejny poziom debilizmu przebity.
|0|(1.0, 1.2898881777401544e-41)
@anonymized_account A niektórzy nadal się cieszą o popierają
|0|(1.0, 5.693567885199322e-34)
można gdzieś w Polsce obejrzeć the bachelor?
|0|(1.0, 3.944370072371776e-33)
@anonymized_account Nie da się 30 meczów w lidze rozegrać pięknych, ważne że te \" brzydkie \" są z punktami
|0|(1.0, 2.8906513677593906e-69)
@anonymized_account @anonymized_account Gdzie tam patrzysz ładniejszy motor ;-/
|0|(1.0, 1.426558517903568e-41)
@anonymized_account A nie wszystkie wiadomości będą wymagane na egzaminie
|0|(1.0, 1.943758510436133e-35)
@anonymized_account A kiedy napiszecie  ile HGW kamienic przewaliła w Warszawie?
|0|(1.0, 8.504223509935

|0|(1.0, 2.7598174879066725e-34)
@anonymized_account Wychodziłoby na to, że Stany są teraz w Unii Europejskiej. Ale skoro Australia może być na Eurowizji...
|0|(1.0, 5.549553668117822e-69)
@anonymized_account Przecież publicznie oświadczył jakiś czas temu że idzie do WOT....
|0|(1.0, 4.836873662885818e-30)
@anonymized_account no to to miała w KrK :)
|0|(1.0, 1.3582893834221163e-19)
@anonymized_account Oczywiście chodzi mi kluby gdzie pieniędzy niebrakuje na trenera
|0|(1.0, 1.1340965483567353e-37)
Błagam nic o zakazie handlu, to tak jak z tym człekiem co patrzy w górę, a za chwilę tłumek
|0|(1.0, 2.060365460349379e-85)
14:45, ale koszyków nadal nie ma 😂
|0|(1.0, 6.355368611879669e-26)
@anonymized_account @anonymized_account Powinni jeszcze sadzać za oszustwa prasowe i durne wypowiedzi w mediach
|0|(1.0, 8.131753650118593e-54)
RT @anonymized_account @anonymized_account @anonymized_account Powinni jeszcze sadzać za oszustwa prasowe i durne wypowiedzi w mediach
|0|(1.0, 1.5545320112045756

@anonymized_account @anonymized_account Ta Wisła jest taka 5 min grają a legia 85 i to legia jest zadowolona z remisu ;)
|0|(1.0, 2.926558473894404e-94)
@anonymized_account Tak :) A potem ten go chyba odepchnął za co dostał 5 meczów :)
|0|(1.0, 1.2503518870785428e-35)
@anonymized_account To jest chore, wszyscy o @anonymized_account
|0|(1.0, 3.2685358615164065e-17)
@anonymized_account @anonymized_account na obu stronach chyba gra :)
|0|(1.0, 5.173083756988693e-35)
@anonymized_account Proszę poczytać wpisy tutaj. Polactwo w pełnej krasie.
|0|(1.0, 1.1024660837327343e-49)
@anonymized_account @anonymized_account Dokładnie tak będzie. I to wcale nie przez to, że PiS taki sprawny a dlatego, że Polacy to debile.
|1|(0.3470640397829321, 0.6529359602170679)
@anonymized_account Zbawiciele świata, poniewierani przez wszystkich, musimy ich chronić
|0|(1.0, 1.4702847629191068e-50)
@anonymized_account przecież ten klub to AntyWisła więc czego Pan wymaga ?
|0|(1.0, 2.0678133275706177e-38)
@anonymized

...

@anonymized_account @anonymized_account Nigdy. Te jego rurki były tak ciasne, że nie jest w stanie ich ściągnąć 😂
|0|(1.0, 7.70251925507334e-59)
@anonymized_account @anonymized_account To nic nowego, że część Polaków od wieków żyje oderwana od rzeczywistości.
|0|(1.0, 1.4164601370215684e-55)
@anonymized_account Nie prościej byłoby kupić drugi ?😁😁😁
|0|(1.0, 1.1279656043224755e-32)
@anonymized_account Tylko, jak pamiętać, które to były? 🤔😁
|0|(1.0, 1.0320431533038705e-41)
@anonymized_account @anonymized_account A skąd Pan wie, jak jeżdżę? 😀
|0|(1.0, 4.430988815549263e-27)
@anonymized_account Wypad parchu z mojej rozmowy. Nie rozmawia z prymitywami o IQ na poziomie temperatury własnego ciała.
|1|(0.0, 1.0)
@anonymized_account @anonymized_account Na podstawie livescore'a też można krótką relację napisać, jak się umie posługiwać językiem polskim.
|0|(1.0, 5.081269847711497e-82)
21 października wytępmy to peowskie robactwo z naszego domu. Że posłużę się retoryką narzuconą przez miłujacą pokó

|0|(1.0, 6.142517460818241e-95)
@anonymized_account Synku gówno widziałeś i gówno wiesz. I nie.mow.o rozchwianie emocjonalnym, bo sam.jestes jak dziecko.
|1|(0.0, 1.0)
#NAME?
|0|(0.9999999999073993, 9.260065287499439e-11)
@anonymized_account @anonymized_account @anonymized_account @anonymized_account Tępym chujem to jesteś ty. Jak nie wierzysz to popatrz w lustro.
|1|(8.832238274081305e-10, 0.9999999991167762)
@anonymized_account @anonymized_account Ordynarna pazerność wladzy, a co na to @anonymized_account alias #beatawystarczyniekraść?
|0|(1.0, 6.895994828801223e-45)
Mam soczek do herbatki od babci love
|0|(1.0, 4.678999185404841e-41)
@anonymized_account dziwne bo patrze na twoje tweety i tylko Wisła i Wisła.To się dopiero nazywają kompleksy.
|0|(1.0, 4.817127893406703e-73)
@anonymized_account No kurwa, zaraz ustawisz mi repertuar, Arek
|0|(1.0, 1.4923759513841759e-33)
@anonymized_account też go nie trawie :)
|0|(1.0, 3.899807518005274e-17)
Przez te puszki po piwie jestem największym

@anonymized_account Won niemcu. Sprzedawaliscie wszystko jak leciało a teraz wielcy patrioci. Won!
|1|(0.0, 1.0)
RT @anonymized_account @anonymized_account Won niemcu. Sprzedawaliscie wszystko jak leciało a teraz wielcy patrioci. Won!
|1|(0.0, 1.0)
@anonymized_account A ta pani to ta słynna dziennikarka inteligentna inaczej @anonymized_account Gargas?
|0|(1.0, 1.2582609090379983e-36)
Nowe płyty L.U.C , donGuralesko, Król, Happysad jest co sluchać
|0|(1.0, 1.7186012917438055e-58)
@anonymized_account PiSkomuna - kurwa, zdzira!\nChciwie w kieszeń Ci spozira\nI wyciąga z niej dukaty. \nChce tu Moskwę mieć. Na raty!
|1|(0.0, 1.0)
sezon na \n\ndziunia w klasie: *kicha*\nwszyscy: NA ZDROWIE OMG DAĆ CI KOCYK?\n\nja: *kicha*\nwszyscy: możesz ciszej?\n\nuważam za otwarty
|0|(1.0, 8.106896466483931e-115)
RT @anonymized_account sezon na \n\ndziunia w klasie: *kicha*\nwszyscy: NA ZDROWIE OMG DAĆ CI KOCYK?\n\nja: *kicha*\nwszyscy: możesz ciszej?\n\nuważa…
|0|(1.0, 1.5356912001889485e-106)
@anonymize

|0|(1.0, 3.086688429452304e-38)
RT @anonymized_account Steve Mandanda is crazy 😂\nhttps://t.co/47B5plJKo1
|0|(1.0, 5.900764064745704e-40)
@anonymized_account @anonymized_account Zalewska chętnie doi fundusze charytatywne z PCK na swoją kampanię wyborczą.
|0|(1.0, 2.339246098069776e-70)
@anonymized_account @anonymized_account @anonymized_account @anonymized_account @anonymized_account co ma Dolha do tego ? Nie było żadnej sprawy o taki kontrakt
|0|(1.0, 4.073422147452886e-37)
@anonymized_account @anonymized_account A jest? Popatrz do słownika i sprawdź znaczenie słów.
|0|(1.0, 3.0185668504075904e-33)
@anonymized_account @anonymized_account A dlaczego miałby być niepłacony ?
|0|(1.0, 1.1295390251119026e-26)
@anonymized_account Zmarnowane pieniadze z majątku osobistego, tak musi być, narmalna sprawiedliwość
|0|(1.0, 1.9299130785509365e-60)
@anonymized_account jest moc. Dziękuję i wzajemnie 😉
|0|(1.0, 1.0446697236887823e-40)
@anonymized_account Można gdzieś w internecie oglądnąć powtórkę ?

@anonymized_account Popyt podaż, ale kur wie lepiej
|1|(3.3639757646142243e-14, 0.9999999999999664)
Jagiellonia jako jedyna w Ekstraklasie w tym roku kalendarzowym wywalczyła dotąd 16 zwycięstw oraz 50 punktów.
|0|(1.0, 1.6659716380778472e-81)
@anonymized_account @anonymized_account No to będzie wtedy walkower.
|0|(1.0, 5.262652503710768e-18)
@anonymized_account @anonymized_account więcej ludzi kojarzy Ronnie O'Sullivana niż jakiegokolwiek futsalowca.Taka prawda.
|0|(1.0, 4.167239780932739e-52)
@anonymized_account @anonymized_account Są trunki 40% gdzie przy minus 30 mi na meczu niebedzie zimno 😀
|0|(1.0, 3.2039866784898824e-58)
@anonymized_account widze że byliście bezlitośni dla przeciwnika :)
|0|(1.0, 3.776423363647467e-43)
Biało-Czerwoni brawo, brawo, brawo! Zbigniew Boniek i Adam Nawałka - wyrazy szacunku. Robert Lewandowski - wielkie podziękowania! 😁😝😎
|0|(1.0, 3.9191999642379566e-131)
RT @anonymized_account Biało-Czerwoni brawo, brawo, brawo! Zbigniew Boniek i Adam Nawałka - wyr

## Training data distribution

Dividing data into segments so that we have both training data and test data

In [153]:
def distributeData(data, ratio):
    counter = ratio * len(data)
    trainingData = []
    testData = []
    for i in range(0, len(data)):
        if i <= counter:
            trainingData.append(data[i])
        else:
            testData.append(data[i])
    return (trainingData, testData)

def randDistributeData(data, ratio):
    random.shuffle(data)
    return distributeData(data, ratio)